In [65]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,TimeoutException
import os
import pandas as pd
from time import sleep
from urllib.parse import urlparse, parse_qs
import csv
import shutil
from pathlib import Path
import time


import json

In [66]:
def backupfiles(backup, nombre_archivo):

    # Leer solo la columna "savedfiles" del archivo CSV
    saved_files = pd.read_csv(backup)["savedfiles"]

    # Verificar si el nuevo archivo está presente en la columna "savedfiles"
    if nombre_archivo in saved_files.values:
        return True
    else:
        return False


In [67]:
def select_doc(driver):
    element = driver.find_element(By.XPATH, '//*[@id="DdlFormat"]/option[1]')
    driver.execute_script("arguments[0].innerHTML = 'excel'", element)
    driver.execute_script("arguments[0].setAttribute('value', '2')", element)


In [68]:
def download_doc(driver):
    driver.find_element(By.ID, 'Button1').click()


In [69]:
def getselects(option=''):
    with open('selects.json', 'r') as archivo_json:
        response = json.load(archivo_json)
    if option:
        return response[option]
    else:
        return response

In [70]:
def seasons(driver, options='0'):

    # get json
    selects = getselects('seasons')

    if  isinstance(options, int):

        season_name = selects[str(options)]
        option = driver.find_element(By.ID, f'TreeViewSeasonn{options}CheckBox')
        option.click()
        print(f"selected seasons: {season_name}")

    elif isinstance(options, str):

        for option in options.split(','):
            option = option.strip()

            if option.isdigit():
                season_name = selects[option]
                check_option = driver.find_element(By.ID, f'TreeViewSeasonn{option}CheckBox')
                check_option.click()
                print(f"selected seasons: {season_name}")
            else:
                season_code = selects[option.title()]
                check_option = driver.find_element(By.ID, f'TreeViewSeasonn{season_code}CheckBox')
                check_option.click()
                print(f"selected seasons: {option}")


In [71]:
def year_pick(driver, options):

     # get json
    selects = getselects('years')

    if isinstance(options, int):

        year_code = selects[option]
        year = driver.find_element(By.ID, f'TreeViewYearn{year_code}CheckBox')
        year.click()
        print(f"selected year: {options}")

    elif isinstance(options, str):

        for option in options.split(','):
            option = option.strip()

            year_code = selects[option]
            year = driver.find_element(By.ID, f'TreeViewYearn{year_code}CheckBox')
            year.click()

            print(f"selected year: {option}")

In [72]:
def districtmode(driver):

    year = driver.find_element(By.ID, 'RadioButtonList1_1')
    year.click()

    print(f"selected districts")

In [73]:
def state_dist_on(driver, options):

    selects = getselects('state_dist_on')

    for option in options.split(','):

        state = driver.find_element(By.ID, f'{selects[option.strip()]["state_id"]}')
        state.click()

    print(f"selected state: {options}")

In [74]:
def crops_pick(driver, options):

    selects = getselects('crops')

    for option in options.split(','):

        state = driver.find_element(By.ID, f'{selects[option.strip()]["id"]}')
        state.click()

    print(f"selected crops: {options}")

In [75]:
def scraper():


    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--start-fullscreen")
    options.add_argument("window-position=-1920,0")

    service = Service(ChromeDriverManager().install())
    url = "https://aps.dac.gov.in/APY/Public_Report1.aspx"


    output= 'output'
    if not os.path.exists(output):
        os.makedirs(output)

    backup = 'datamemory.csv'
    if not os.path.exists(backup):
        with open(backup, mode = 'w', newline='') as file:
          file.write('savedfiles\n')

    response = getselects()
    years = response['years']
    states = response['state_dist_on']
    crops = response['crops']

    maximum_scraper_time = 1800
    init_time = time.time()

    for year in years:
        yearname = year
        for state_id, state in states.items():
            statename = state['name']
            for crop_id, crop in crops.items():
                cropsname = crop['name']

                if time.time() - init_time > maximum_scraper_time:
                    time.sleep(300)

                filename_beta = f'year_{yearname}_st_{statename}_crop_{cropsname}.xls'
                filename = filename_beta.replace(' ', '_').replace('/', '_')

                if backupfiles(backup, filename):
                    continue

                driver = webdriver.Chrome(service=service, options=options)
                driver.get(url)

                districtmode(driver)

                seasons(driver)

                year_pick(driver,year)

                state_dist_on(driver,state_id)

                crops_pick(driver, crop_id)

                select_doc(driver)

                print(filename)

                download_doc(driver)
                # ++++++++Start download++++++++++

                # carpeta destino
                carpeta_destino = "output"

                # Nombre del archivo que esperas
                # filename

                # nombre de archivo por defecto
                defaulname = 'Report.xls'

                # Obtener el user profile folder
                downloads_path = Path.home() / "Downloads"

                # Ruta completa de los archivos
                ruta_archivo_origen = os.path.join(downloads_path, defaulname)
                ruta_archivo_destino = os.path.join(carpeta_destino, filename)

                # Tiempo de espera máximo en segundos
                tiempo_maximo_espera = 60

                while not os.path.exists(os.path.join(downloads_path, defaulname)):
                    sleep(0.5)

                # Obtener el tamaño inicial del archivo
                tamaño_anterior  = os.path.getsize(ruta_archivo_origen)

                # Inicializar el tiempo de inicio
                tiempo_inicio = time.time()
                # Esperar a que el archivo termine de descargar
                while True:
                    # Verificar si el tamaño del archivo sigue siendo el mismo
                    tamaño_actual = os.path.getsize(ruta_archivo_origen)
                    if tamaño_actual == tamaño_anterior :
                        # El tamaño del archivo no ha cambiado, lo que indica que ha terminado de descargar
                        break
                        # Verificar si se ha excedido el tiempo máximo de espera
                    if time.time() - tiempo_inicio > tiempo_maximo_espera:
                        print("Se ha excedido el tiempo de espera máximo.")
                        break
                        # Actualizar el tamaño anterior para la próxima iteración
                    tamaño_anterior = tamaño_actual
                    # Esperar un breve período antes de volver a verificar
                    time.sleep(0.5)
                # Mover el archivo a la nueva carpeta
                shutil.move(ruta_archivo_origen, ruta_archivo_destino)

                # Verificar si el archivo se movió correctamente
                if os.path.exists(ruta_archivo_destino):
                    print("El archivo se movió correctamente.")
                else:
                    print("Hubo un error al mover el archivo.")
                                # ++++++++End download++++++++++++

                with open(backup, mode = 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([filename])


                driver.quit()






In [76]:
scraper()

selected districts
selected seasons: All
selected year: 1997
selected state: 1
selected crops: 1
1997_Andaman_and_Nicobar_Islands_Arecanut.xls
El archivo se movió correctamente.
selected districts
selected seasons: All
selected year: 1997
selected state: 1
selected crops: 2
1997_Andaman_and_Nicobar_Islands_Arhar_Tur.xls
El archivo se movió correctamente.
selected districts
selected seasons: All
selected year: 1997
selected state: 1
selected crops: 3
1997_Andaman_and_Nicobar_Islands_Bajra.xls
El archivo se movió correctamente.
selected districts
selected seasons: All
selected year: 1997
selected state: 1
selected crops: 4
1997_Andaman_and_Nicobar_Islands_Banana.xls
El archivo se movió correctamente.


KeyboardInterrupt: 